In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import missingno as msno
from collections import Counter
from itertools import chain, combinations
import sklearn as sk

# SVM

In [4]:
data0 = pd.read_csv("newdata3.csv", engine='python')
data0.head()

,Unnamed: 0,weight_16,height_16,iq,dep_band_15,anx_band_15,exercise,child_bull,has_dep_diag,secd_diag,...,read,text,tran,work,comp,tv,talk_mob,talk_phon,comp_bed_16,comp_house_at_all
0,0,59.294132,181.602831,107.0,0.5,0.5,4.0,0.0,0.0,0.0,...,2.0,2.0,0.0,6.0,4.0,4.0,2.0,1.0,0.0,1.0
1,1,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,3,49.812426,160.224186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,4,62.270030,191.703227,132.0,0.1,3.0,3.0,0.0,NaN,NaN,...,4.0,0.0,0.0,2.0,3.0,5.0,0.0,0.0,1.0,1.0


In [6]:
data1 = pd.read_csv("imputed_dataset_1.csv", engine='python')
data1.head()
data2 = pd.read_csv("imputed_dataset_2.csv", engine='python')
data2.head()
data3 = pd.read_csv("imputed_dataset_3.csv", engine='python')
data3.head()
data4 = pd.read_csv("imputed_dataset_4.csv", engine='python')
data4.head()
data5 = pd.read_csv("imputed_dataset_5.csv", engine='python')
data5.head()

,Unnamed: 0,weight_16,height_16,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,...,anx_band_15,exercise,child_bull,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house
0,0,59.294132,181.602831,0.0,0.0,1.0,1.0,1.0,2.0,0.0,...,0.5,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1,62.290242,168.453878,0.0,0.0,1.0,1.0,1.0,2.0,0.0,...,3.0,3.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0
2,2,62.290242,168.453878,0.0,0.0,1.0,1.0,1.0,2.0,0.0,...,3.0,3.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0
3,3,49.812426,160.224186,0.0,0.0,1.0,2.0,1.0,2.0,0.0,...,3.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
4,4,62.270030,191.703227,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.0,3.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0


Here you are going to study and discuss SVMs as a model for the fashion MNIST classification dataset. In particular you should:

1. Train an SVM (with a specific Kernel), plot the training and validation learning curves. You may need to subsample the dataset if SVM training is taking too long. Do you see any signs of overfitting? Interpret and discuss your results. (1 mark)
2. What are your results in the testing dataset? Interpret and discuss your results. (2 marks)
3. How sensitive is this method to different hyperparameters? For example the different types of kernel (e.g. linear, RBF, etc.). Make use of plots (e.g. performance on test dataset as a function of different hyperparameters) to help you discuss this point. (5 marks)
4. Plot decision boundaries and discuss their relevance. (2 marks)
5. Compare your SVM results with the ANN above in terms of performance and the time it takes to train each method. For example, use bar plots to compare their performances and training times next to each other. Which is the better model? And why? (5 marks)


In [8]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13734 entries, 0 to 13733
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         13734 non-null  int64  
 1   weight_16          13734 non-null  float64
 2   height_16          13734 non-null  float64
 3   comp_noint_bed_16  13734 non-null  float64
 4   comp_int_bed_16    13734 non-null  float64
 5   talk_phon_wend     13734 non-null  float64
 6   text_wend          13734 non-null  float64
 7   talk_mob_wend      13734 non-null  float64
 8   comp_wend          13734 non-null  float64
 9   musi_wend          13734 non-null  float64
 10  read_wend          13734 non-null  float64
 11  work_wend          13734 non-null  float64
 12  alon_wend          13734 non-null  float64
 13  draw_wend          13734 non-null  float64
 14  play_wend          13734 non-null  float64
 15  tv_wend            13734 non-null  float64
 16  out_win_wend       137

In [16]:
Y = np.column_stack((data1['has_dep_diag'], data1['secd_diag'], data1['prim_diag'], data1['dep_score'], data1['dep_thoughts']))
print(Y.shape)

(13734, 5)


In [17]:
X = data1.drop(['has_dep_diag'],axis=1).drop(['secd_diag'],axis=1).drop(['prim_diag'],axis=1).drop(['dep_score'],axis=1).drop(['dep_thoughts'],axis=1)
print(X.shape)

(13734, 38)


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.1, random_state=40)
print(X_train.shape); print(X_test.shape)
print(y_train.shape); print(y_test.shape)


(12360, 38)
(1374, 38)
(12360, 5)
(1374, 5)


In [22]:
##to shuffle!!!

##np.random.seed(42) 
##shuffle_index = np.random.permutation(12360)
##X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]
##np.random.seed(42)
##shuffle_index_test = np.random.permutation(1374)
##X_test, y_test = X_test[shuffle_index_test], y_test[shuffle_index_test]

#reduce the dataset
X_train_reduced, y_train_reduced = X_train[0:1000], y_train[0:1000]
X_test_reduced, y_test_reduced = X_test[0:200], y_test[0:200]
print(X_train_reduced.shape); print(X_test_reduced.shape)
print(y_train_reduced.shape); print(y_test_reduced.shape)

(1000, 38)
(200, 38)
(1000, 5)
(200, 5)


In [23]:
from sklearn import svm
from sklearn.svm import SVC

In [26]:
svm = SVC(C=4)
svm.fit(X_train, y_train[:,0])
print(f'SVM classifier accuracy: {svm.score(X_train, y_train[:,0])}')
print(f'SVM classifier accuracy - test set: {svm.score(X_test, y_test[:,0])}')

SVM classifier accuracy: 0.9713592233009709
SVM classifier accuracy - test set: 0.9745269286754003


In [27]:
svm = SVC(C=4)
svm.fit(X_train, y_train[:,1])
print(f'SVM classifier accuracy: {svm.score(X_train, y_train[:,1])}')
print(f'SVM classifier accuracy - test set: {svm.score(X_test, y_test[:,1])}')

SVM classifier accuracy: 0.961084142394822
SVM classifier accuracy - test set: 0.9621542940320232


In [28]:
svm = SVC(C=4)
svm.fit(X_train, y_train[:,2])
print(f'SVM classifier accuracy: {svm.score(X_train, y_train[:,2])}')
print(f'SVM classifier accuracy - test set: {svm.score(X_test, y_test[:,2])}')

SVM classifier accuracy: 0.9312297734627831
SVM classifier accuracy - test set: 0.9344978165938864


In [29]:
svm = SVC(C=4)
svm.fit(X_train, y_train[:,3])
print(f'SVM classifier accuracy: {svm.score(X_train, y_train[:,3])}')
print(f'SVM classifier accuracy - test set: {svm.score(X_test, y_test[:,3])}')

SVM classifier accuracy: 0.9222491909385113
SVM classifier accuracy - test set: 0.9352256186317321


In [30]:
svm = SVC(C=4)
svm.fit(X_train, y_train[:,4])
print(f'SVM classifier accuracy: {svm.score(X_train, y_train[:,4])}')
print(f'SVM classifier accuracy - test set: {svm.score(X_test, y_test[:,4])}')

SVM classifier accuracy: 0.9325242718446602
SVM classifier accuracy - test set: 0.9461426491994177


In [33]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1,1, 10, 100, 1000], 
    'gamma': [1,0.1,0.01,0.001, 0.0001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}
tuned_svm = GridSearchCV(svm, param_grid, cv = 5, scoring='accuracy', verbose = 2, n_jobs=-1)
tuned_svm.fit(X_train_reduced, y_train_reduced[:,0])

params = tuned_svm.best_params_

print(params)
print(tuned_svm.score(X_train,y_train[:,0]))
print(tuned_svm.score(X_test,y_test[:,0]))

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 